Lorenzo Colletta, Giorgio Fantilli, Luca Lucioli

## Parte 1 - Descrizione del problema

Il dataset in esame include dati per la stima dei livelli di obesità negli individui provenienti dai paesi di Messico, Perù e Colombia, in base alle loro abitudini alimentari e alle condizioni fisiche.
Le istanze sono classificate in base al livello di obesità (Peso Insufficiente, Peso Normale, Sovrappeso Livello I, Sovrappeso Livello II, Obesità Tipo I, Obesità Tipo II e Obesità Tipo III). Il 77% dei dati è stato generato artificialmente utilizzando lo strumento Weka e il filtro SMOTE, mentre il 23% dei dati è stato raccolto direttamente dagli utenti attraverso una piattaforma web.

L'obiettivo è predire il livello di obesità dei soggetti utilizzando i loro dati antropometrici, le informazioni sulle loro abitudini alimentari e sull'attività fisica.

Di seguito vengono importate le librerie necessarie per scaricare i file, organizzare le strutture dati e disegnare i grafici.

In [ ]:
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dython.nominal import associations, cramers_v
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

### Caricamento dati e preprocessing

In [ ]:
file_zip_url = "https://archive.ics.uci.edu/static/public/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition.zip"
file_zip_name = "obesity_data.zip"
file = "./ObesityDataSet_raw_and_data_sinthetic.csv"

if not os.path.exists(file_zip_name):
    from urllib.request import urlretrieve
    urlretrieve(file_zip_url, file_zip_name)
    from zipfile import ZipFile
    with ZipFile(file_zip_name) as f:
        f.extractall()

obesity_raw_data = pd.read_csv(file)
obesity_raw_data = obesity_raw_data.rename(columns={"family_history_with_overweight" : "FHWO", "NObeyesdad" : "ObesityLevel"})

obesity_raw_data.head(10)

Sopra viene mostrato un piccolo estratto del dataset in esame, dopo aver rinominato alcune feature per migliorarne la leggibilità e l'utilizzo.

Di seguito sono riportate le dimensioni in memoria, il numero di istanze non nulle e il tipo delle feature che compongono i dati raccolti nel dataset.


In [ ]:
obesity_raw_data.info(memory_usage="deep")

Osserviamo come molte delle feature sono di tipo `object`. Procediamo di seguito con la trasformazione in variabili categoriche e booleane per limitare la memoria occupata.

In [ ]:
categorical = ["Gender", "CAEC", "CALC", "MTRANS", "ObesityLevel"]
boolean = ["FHWO", "FAVC", "SMOKE", "SCC"]

obesity_raw_data[categorical] = obesity_raw_data[categorical].astype("category")

obesity_raw_data[boolean] = obesity_raw_data[boolean].replace({"yes": True, "no": False})

obesity_raw_data.info(memory_usage="deep")

Osserviamo come tutte le feature sono rileventi per il problema, e soprattutto che nessuna delle stesse presenta valori nulli. Non risulta perciò necessaria nessuna pulizia dei dati.

### Descrizione delle feature
Il dataset contiene le seguenti feature:
*   `Gender`: genere (Male, Female)
*   `Age`: età
*   `Height`: altezza in metri
*   `Weight`: peso in kilogrammi
*   `FHWO`: indica se il soggetto ha familiari che sono o sono stati sovrappeso (yes, no)
*   `FAVC`: indica se il soggetto mangia spesso cibo ad alto contenuto calorico o meno (yes, no)
*   `FCVC`: indica se solitamente il soggetto mangia verdura durante i pasti o meno (Never, Sometimes, Always)
*   `NCP`: indica quanti pasti consuma il soggetto durante la giornata (tra 1 e 2, 3, più di 3)
*   `CAEC`: indica se il soggetto mangia tra i pasti (No, Sometimes, Frequently, Always)
*   `SMOKE`: indica se il soggetto fuma o meno (yes, no)
*   `CH2O`: indica quanta acqua beve il soggetto durante il giorno (meno di 1l, tra 1l e 2l, più di 2l)
*   `SCC`: indica se il soggetto monitora le calorie che assume durante il giorno (yes, no)
*   `FAF`: indica quanto spesso il soggetto svolge attività fisica durante la settimana (mai, 1 o 2 giorni, 2 o 4 giorni, 4 o 5 giorni)
*   `TUE`: indica quanto spesso il soggetto utilizza apparecchi tecnologici durante il giorno come telefono, videogiochi, tv, computer o altri (0-2 ore, 3-5 ore, più di 5 ore)
*   `CALC`: indica quanto spesso il soggetto beve alcool (Never, Sometimes, Frequently, Always)
*   `MTRANS`: indica quale mezzo di trasporto utilizza solitamente il soggetto (Automobile, Motorbike, Bike, Public Transportation, Walking)

La variabile target è `ObesityLevel`, che rappresenta il livello di obesità nelle seguenti classi: Insufficient Weight, Normal Weight, Overweight Level I, Overweight Level II, Obesity Type I, Obesity Type II, and Obesity Type III.

Le features in esame sono sia categoriche, binarie, che continue:<br>
Tra le categoriche troviamo: `Gender`, `CAEC`, `CALC` e `MTRANS`.<br>
Tra le continue troviamo: `Age`, `Height`, `Weight`, `FCVC`, `NCP`, `CH2O`, `FAF` e `TUE`.<br>
Tra le binarie troviamo: `FHWO`, `FAVC`, `SMOKE` e `SCC`.

Si segnala che `FCVC`, `NCP`, `CH2O`, `FAF` e `TUE` sono feature di natura categorica ma assumono valori continui in quanto una parte dei dati è stata generata artificialmente. Si posticipa la scelta dell'approccio da utilizzare per la gestione di tali variabili.

gender nominale
age continua 
height continua
weight continua
FHWO boolean
FAVC boolean
FCVC ordinale
NCP ordinale
CAEC ordinale
SMOKE boolean
CH2O ordinale
SCC boolean
FAF ordinale
TUE ordinale
CALC ordinale
MTRANS nominale

In [ ]:
obesity_raw_data["ObesityLevel"].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
plt.ylabel(None)
plt.figure().set_facecolor('white')
plt.show()

obesity_raw_data["ObesityLevel"].value_counts().plot.bar(color='skyblue', rot=90, xlabel='Livello di Obesità', ylabel='Numero di Istanze', title='Distribuzione delle Istanze per Livello di Obesità')

Osserviamo dai grafici come la variabile target `ObesityLevel` risulta essere bilanciata, ovvero il numero di istanze per le possibili classi sono pressochè identiche.  Il problema in esame risulta quindi essere bilanciato e non e' necessario utilizzare tecniche di bilanciamento delle classi.

In [ ]:
obesity_raw_data.isna().sum()

Si verifica come nessuna delle feature del dataset presenta valori nulli. Osserviamo inoltre che tutte le variabili sono rileventi per la modellazione del problema. Non risulta perciò necessaria nessuna pulizia dei dati.

## Parte 2 - Analisi Esplorativa

### Esplorazione delle feature continue

Di seguito sono riportate le statistiche generali delle feature continue del dataset (media, minimo, massimo, deviazione standard e percentili) e vengono mostrati gli istogrammi relativi alla distribuzione dei loro valori.

In [ ]:
obesity_raw_data[['Age', 'Height', 'Weight']].describe()

Si nota che i soggetti del campione hanno tra i 14 e i 61 anni. Rilevando che il 75% ha meno di 26 anni, la media è di circa 24 anni e la deviazione standard è 6.35, si può ipotizzare che la maggior parte dei soggetti ha tra i 20 e i 30 anni.<br>
Il peso massimo registrato è pari a 173kg ma il 75% dei soggetti pesa meno di 107kg, è perciò probabile la presenza di valori di peso particolari nell'estremo superiore. Nell'estremo inferiore i valori sono invece coerenti con le caratteristiche antropometriche del campione (età e altezza).

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.title('Age')
plt.hist(obesity_raw_data['Age'], label="Age", color='gray')
plt.ylabel('count')
plt.xlabel('anni')

plt.subplot(1, 3, 2)
plt.title('Weight')
plt.hist(obesity_raw_data['Weight'], label="weight")
plt.ylabel('count')
plt.xlabel('Kg')

plt.subplot(1, 3, 3)
plt.title('Height')
plt.hist(obesity_raw_data['Height'], label="height", color="tab:red")
plt.ylabel('count')
plt.xlabel('m')
plt.show()

* L'istogramma dell'età conferma l'ipotesi di cui sopra: la maggior parte dei soggetti ha un età compresa tra i 20 e i 30 anni. Si può osservare come il numero delle istanze tenda a diminuire all'avanzare dell'età, diventando trascurabile nella parte finale del grafico.
* Come precedentemente osservato, il grafico del peso rileva una distribuzione delle istanze congrua con il campione, ad eccezione dei valori più alti che risultano poco popolati.
* Il grafico dell'altezza è conforme alla distribuzione attesa per la variabile. 

#### Rimozione degli outliers

In [ ]:
plt.figure(figsize=(20, 30))

plt.subplot(4, 3, 1)
plt.title('Age')
plt.boxplot(obesity_raw_data['Age'])
plt.ylabel('anni')

plt.subplot(4, 3, 2)
plt.title('Weight')
plt.boxplot(obesity_raw_data['Weight'])
plt.ylabel('Kg')

plt.subplot(4, 3, 3)
plt.title('Height')
plt.boxplot(obesity_raw_data['Height'])
plt.ylabel('m')

plt.show()

I grafici sopra permettono di visualizzare meglio la distribuzione dei dati più estremi del campione.<br>
Il primo box plot conferma la tendenza dei record a diminuire con l'aumentare dell'età, in particolare il numero di istanze che assumono un'età superiore a circa 50 anni diventa trascurabile.<br>
Il secondo grafico mostra la presenza di istanze che assumono valori massimi anomali nell'estremità superiore per la variabile del peso, come precedentemente ipotizzato.<br>
La distribuzione delle altezze si conferma corretta, a scanso di isolati valori limite superiori.

Si procede di seguito, in base alle precedenti osservazioni, con la rimozione degli outliers sulla base delle deviazioni standard.

In [ ]:
obesity_raw_data.drop(obesity_raw_data.loc[abs(obesity_raw_data['Age'] - obesity_raw_data['Age'].mean()) > 3 * obesity_raw_data['Age'].std()].index, inplace=True)
obesity_raw_data.drop(obesity_raw_data.loc[abs(obesity_raw_data['Weight'] - obesity_raw_data['Weight'].mean()) > 3 * obesity_raw_data['Weight'].std()].index, inplace=True)
obesity_raw_data.drop(obesity_raw_data.loc[abs(obesity_raw_data['Height'] - obesity_raw_data['Height'].mean()) > 3 * obesity_raw_data['Height'].std()].index, inplace=True)

In [ ]:
plt.figure(figsize=(20, 30))

plt.subplot(4, 3, 1)
plt.title('Age')
plt.boxplot(obesity_raw_data['Age'])
plt.ylabel('anni')

plt.subplot(4, 3, 2)
plt.title('Weight')
plt.boxplot(obesity_raw_data['Weight'])
plt.ylabel('Kg')

plt.subplot(4, 3, 3)
plt.title('Height')
plt.boxplot(obesity_raw_data['Height'])
plt.ylabel('m')

plt.show()

In [ ]:
obesity_raw_data[['Age', 'Height', 'Weight']].describe()

L'operazione comporta la perdita di sole 25 istanze limite. Dai grafici e dalle statistiche generali delle feature si osserva che sono state rimossi i record con età superiore ai circa 44 anni, con un peso superiore ai circa 165kg; non sono stati cancellati record sulla base dei valori assunti dall'altezza.<br>
Le colonne interessate hanno mantenuto la medesima media, andando invece a migliorare la relativa deviazione standard.

### Esplorazione delle feature continue di natura categorica

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(2, 3, 1)
plt.title('FCVC')
fcvc = obesity_raw_data["FCVC"].value_counts()
plt.scatter(fcvc.keys(), fcvc.values, color="purple")
plt.ylabel('count')
plt.xlabel('frequenza')

plt.subplot(2, 3, 2)
plt.title('NCP')
ncp = obesity_raw_data["NCP"].value_counts()
plt.scatter(ncp.keys(), ncp.values)
plt.ylabel('count')
plt.xlabel('n. pasti')


plt.subplot(2, 3, 3)
plt.title('CH2O')
ch2o = obesity_raw_data["CH2O"].value_counts()
plt.scatter(ch2o.keys(), ch2o.values, color="tab:orange")
plt.ylabel('count')
plt.xlabel('l')

plt.subplot(2, 3, 4)
plt.title('FAF')
faf = obesity_raw_data["FAF"].value_counts()
plt.scatter(faf.keys(), faf.values, color="tab:red")
plt.ylabel('count')
plt.xlabel('giorni a settimana')

plt.subplot(2, 3, 5)
plt.title('TUE')
tue = obesity_raw_data["TUE"].value_counts()
plt.scatter(tue.keys(), tue.values, color="tab:green")
plt.ylabel('count')
plt.xlabel('ore giornaliere')

plt.show()

I grafici mostrati sono relativi alle sopracitate variabili di natura categorica ma trattate come continue. Si osserva che la maggior parte delle istanze ricadono precisamente in delle specifiche classi, mentre i restanti valori si distribuiscono in maniera uniforme tra le stesse.
Vista la necessità di ricondursi a valori di tipo categorico si procede con l'arrotondamento al valore intero più vicino.

In [ ]:
obesity_round_data = obesity_raw_data.copy()
obesity_round_data['NCP'] = obesity_round_data['NCP'].apply(lambda x: 1 if 1 <= x < 2 else (3 if 2 <= x < 3.5 else round(x))).astype(int) 
obesity_round_data['FCVC'] = obesity_round_data['FCVC'].round().astype(int)
obesity_round_data['CH2O'] = obesity_round_data['CH2O'].round().astype(int)
obesity_round_data['FAF'] = obesity_round_data['FAF'].round().astype(int)
obesity_round_data['TUE'] = obesity_round_data['TUE'].round().astype(int)

plt.figure(figsize=(20, 10))

plt.subplot(2, 3, 1)
plt.title('FCVC')
fcvc = obesity_round_data["FCVC"].value_counts()
plt.scatter(fcvc.keys(), fcvc.values, color="purple")
plt.ylabel('count')
plt.xlabel('frequenza')

plt.subplot(2, 3, 2)
plt.title('NCP')
ncp = obesity_round_data["NCP"].value_counts()
plt.scatter(ncp.keys(), ncp.values)
plt.ylabel('count')
plt.xlabel('n. pasti')


plt.subplot(2, 3, 3)
plt.title('CH2O')
ch2o = obesity_round_data["CH2O"].value_counts()
plt.scatter(ch2o.keys(), ch2o.values, color="tab:orange")
plt.ylabel('count')
plt.xlabel('l')

plt.subplot(2, 3, 4)
plt.title('FAF')
faf = obesity_round_data["FAF"].value_counts()
plt.scatter(faf.keys(), faf.values, color="tab:red")
plt.ylabel('count')
plt.xlabel('giorni a settimana')

plt.subplot(2, 3, 5)
plt.title('TUE')
tue = obesity_round_data["TUE"].value_counts()
plt.scatter(tue.keys(), tue.values, color="tab:green")
plt.ylabel('count')
plt.xlabel('ore giornaliere')

plt.show()

Sopra sono riportati i grafici delle feature dopo l'arrotondamento.

Di seguito le statistiche generali delle variabili prima e dopo l'operazione. 

In [ ]:
obesity_raw_data[['NCP', 'FCVC', 'CH2O', 'FAF', 'TUE']].describe()

In [ ]:
obesity_round_data[['NCP', 'FCVC', 'CH2O', 'FAF', 'TUE']].describe()

Come si può notare, riportare i valori non interi all'interno delle classi contemplate ha avuto un impatto alquanto limitato sulle statistiche generali delle feature trattate. In particolare media e deviazione standard dimostrano come la distribuzione dei valori sia coerente con la precedene, grazie ad un corretto bilanciamento dei record generati.

### Esplorazione delle feature categoriche

In [ ]:
color= ['tab:blue', 'tab:red', 'tab:green', 'tab:orange', 'tab:purple']
plt.figure(figsize=(20, 20))

plt.subplot(2, 2, 1)
plt.title('Gender')
plt.bar(obesity_raw_data['Gender'].unique(),
        obesity_raw_data['Gender'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 2)
plt.title('CAEC')
plt.bar(obesity_raw_data['CAEC'].unique(),
        obesity_raw_data['CAEC'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 3)
plt.title('CALC')
plt.bar(obesity_raw_data['CALC'].unique(),
        obesity_raw_data['CALC'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 4)
plt.title('MTRANS')
plt.bar(obesity_raw_data['MTRANS'].unique(),
        obesity_raw_data['MTRANS'].value_counts(),
        color=color)
plt.ylabel('count')

plt.show()

Dagli istrogrammi sopra osserviamo che i soggetti sono pressochè equamente distribuiti tra maschi e femmine, mangiano qualche volta tra i pasti, non bevono alcool o molto poco e si muovono principalmente con i trasporti pubblici o a piedi.

### Esplorazione delle feature booleane

In [ ]:
color = ['tab:blue', 'tab:red']
plt.figure(figsize=(20, 20))

plt.subplot(2, 2, 1)
plt.title('FHWO')
plt.bar(obesity_raw_data['FHWO'].replace({True: "SI", False: "NO"}).unique(),
        obesity_raw_data['FHWO'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 2)
plt.title('FAVC')
plt.bar(obesity_raw_data['FAVC'].replace({True: "SI", False: "NO"}).unique(),
        obesity_raw_data['FAVC'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 3)
plt.title('SMOKE')
plt.bar(obesity_raw_data['SMOKE'].replace({True: "SI", False: "NO"}).unique(),
        obesity_raw_data['SMOKE'].value_counts(),
        color=color)
plt.ylabel('count')

plt.subplot(2, 2, 4)
plt.title('SCC')
plt.bar(obesity_raw_data['SCC'].replace({True: "SI", False: "NO"}).unique(),
        obesity_raw_data['SCC'].value_counts(),
        color=color)
plt.ylabel('count')

plt.show()

I grafici mostrano un comportamento salubre della maggior parte del campione, essendo composto da pochi fumatori e pochi soggetti che mangiano regolarmente alimenti ad alto contenuto calorico (FAVC). La maggioranza non ha familiari che sono o sono stati sovrappeso e non monitora le calorie che assume.

### Indici di correlazione

In [ ]:
correlation = associations(
    obesity_round_data, 
    nominal_columns='auto',
    numerical_columns=None, 
    mark_columns=False, 
    nom_nom_assoc='cramer', 
    num_num_assoc='spearman', 
    nan_strategy='replace',
    plot=False
)

In [ ]:
def plot_correlation(dataset):
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    # Generate a mask for the upper triangle
    mask = np.zeros_like(dataset, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(dataset, mask=mask, cmap=cmap, vmax=.5, center=0,annot = True, square=True, linewidths=.5, cbar_kws={"shrink": .5});

plot_correlation(correlation['corr'])

In [ ]:
dataset = obesity_round_data.copy()
dataset['Gender'] = dataset['Gender'].replace({"Female": False, "Male": True})
dataset['CAEC'] = dataset['CAEC'].replace({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3})
dataset['CALC'] = dataset['CALC'].replace({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3})
dataset['MTRANS'] = dataset['MTRANS'].replace({"Automobile": 0, "Motorbike": 1, "Bike": 3, "Public_Transportation": 2, "Walking": 4})
dataset['ObesityLevel'] = dataset['ObesityLevel'].replace({'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4, 'Obesity_Type_II': 5, 'Obesity_Type_III': 6})

plot_correlation(dataset.corr())

continua-continua pearson
continua-ordinale kendall
continua-nominale point-biserial
ordinale-ordinale kendall (spearman se tanti livelli)
ordinale-nominale rank-biserial
nominale-nominale 

In [ ]:
import scipy.stats 
dataset = obesity_round_data.copy()

dataset['CAEC'] = dataset['CAEC'].replace({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3})
dataset['CALC'] = dataset['CALC'].replace({"no": 0, "Sometimes": 1, "Frequently": 2, "Always": 3})

variabili_continue = ['Age', 'Height', 'Weight']
variabili_nominale = ['Gender', 'FHWO', 'FAVC', 'SMOKE', 'SCC', 'MTRANS']
variabili_ordinale = ['FCVC', 'NCP', 'CAEC', 'CH2O', 'FAF', 'TUE', 'CALC']

# combined_correlation = pd.DataFrame(index=variabili_continue + variabili_ordinale + variabili_nominale,
#                                      columns=variabili_continue + variabili_ordinale + variabili_nominale)

# for col1 in variabili_nominale:
#     for col2 in variabili_nominale:
#         if col1 != col2:
#             correlation = cramers_v(dataset[col1], dataset[col2])
#             combined_correlation.loc[col1, col2] = correlation
#             combined_correlation.loc[col2, col1] = correlation

dataset['Gender'] = dataset['Gender'].replace({"Female": 0, "Male": 1})
dataset['MTRANS'] = dataset['MTRANS'].replace({"Automobile": 0, "Motorbike": 1, "Bike": 3, "Public_Transportation": 2, "Walking": 4})
dataset['ObesityLevel'] = dataset['ObesityLevel'].replace({'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I': 2, 'Overweight_Level_II': 3, 'Obesity_Type_I': 4, 'Obesity_Type_II': 5, 'Obesity_Type_III': 6})


# # Calcolo delle correlazioni tra variabili continue
# combined_correlation.loc[variabili_continue, variabili_continue] = dataset[variabili_continue].corr(method='pearson')

# # Calcolo delle correlazioni tra variabili ordinali
# combined_correlation.loc[variabili_ordinale, variabili_ordinale] = dataset[variabili_ordinale].corr(method='kendall')

# # Calcolo delle correlazioni tra variabili nominale e altre variabili
# for col in variabili_nominale:
#     combined_correlation.loc[variabili_continue, col] = [
#         pointbiserialr(dataset[col].astype('int'), dataset[c])[0] for c in variabili_continue
#     ]

#     # Usare solo il coefficiente di point-biserial per la correlazione nominale-ordinale
#     combined_correlation.loc[variabili_ordinale, col] = [
#         pointbiserialr(dataset[col].astype('int'), dataset[c])[0] for c in variabili_ordinale
#     ]

#     combined_correlation.loc[col, variabili_continue + variabili_ordinale] = combined_correlation.loc[variabili_continue + variabili_ordinale, col]



# # Visualizzazione della matrice di correlazione combinata
# print(combined_correlation)
# combined_correlation = combined_correlation.fillna(0)

# # Esegui un casting di tutti i dati a tipo float
# combined_correlation = combined_correlation.astype(float)

# # Creazione di una heatmap con seaborn
# plt.figure(figsize=(12, 10))
# sns.heatmap(combined_correlation, annot=True, cmap='coolwarm', fmt=".2f")
# plt.show()
dataset.notnull

In [ ]:
continuous_columns = ['Age', 'Height', 'Weight']
continuous_data = obesity_round_data[continuous_columns]

correlation_matrix = continuous_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()

In [ ]:
nominal_data = obesity_round_data[variabili_nominale]
combined_correlation = pd.DataFrame(index=variabili_nominale,
                                     columns=variabili_nominale)
for col1 in variabili_nominale:
    for col2 in variabili_nominale:
        if col1 != col2:
            correlation = cramers_v(obesity_round_data[col1], obesity_round_data[col2])
            combined_correlation.loc[col1, col2] = correlation
            combined_correlation.loc[col2, col1] = correlation

print(combined_correlation)

In [ ]:
ordinal_data = obesity_round_data[variabili_ordinale]
matrix1 = dataset[variabili_ordinale].corr(method='kendall')
sns.heatmap(matrix1, annot=True, cmap='coolwarm', vmin=0, vmax=1)
plt.show()

In [ ]:
encoder = OneHotEncoder(sparse=False)
ms = pd.DataFrame(
    encoder.fit_transform(dataset[['MTRANS']]),
    columns=encoder.get_feature_names_out(dataset[['MTRANS']].columns)
)
dataset.drop(columns=['MTRANS'], inplace=True)
dataset.reset_index(drop=True, inplace=True)
ms.reset_index(drop=True, inplace=True)
dataset = pd.concat([dataset, ms], axis=1)
dataset.head()

In [ ]:
dataset

In [ ]:
from scipy.stats import pointbiserialr
variabili_nominale = ['Gender', 'FHWO', 'FAVC', 'SMOKE', 'SCC', 'MTRANS_0', 'MTRANS_1', 'MTRANS_2', 'MTRANS_3', 'MTRANS_4']
# Creiamo una matrice vuota per la correlazione point-biserial
matrice_correlazione = pd.DataFrame(index=variabili_continue, columns=variabili_nominale)

# Calcoliamo la correlazione point-biserial per ogni coppia di variabili
for col_cont in variabili_continue:
    for col_nom in variabili_nominale:
        r, p_value = pointbiserialr(dataset[col_nom], dataset[col_cont])
        matrice_correlazione.loc[col_cont, col_nom] = float(r)

# Visualizziamo la matrice di correlazione point-biserial
print(matrice_correlazione)
plot_correlation(pd.DataFrame(matrice_correlazione))


In [ ]:
matrix3 = dataset[variabili_ordinale + variabili_continue].corr(method='spearman')
print(matrix3)
sns.heatmap(matrix3, annot=True, cmap='coolwarm', vmin=0, vmax=1)
plt.show()